In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

%matplotlib inline
#from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
#import pydotplus

In [ ]:
import matplotlib
matplotlib.__version__

In [ ]:
sns.__version__

In [ ]:
sns.set_style("whitegrid")

In [ ]:
def get_data(week_nums):
    # base url
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        # concat our week onto url..
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [170422, 170429, 170506, 170513]
# Test against Julia's nums
#week_nums = [160903, 160910, 160917]
df = get_data(week_nums)

In [ ]:
df.columns.tolist()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns = [column.strip() for column in df.columns]

In [ ]:
# df.DATE.value_counts().sort_index()

In [ ]:
df['KEY'] = df['C/A'] + ',' + df['UNIT'] + ',' + df['SCP'] + ',' + df['STATION']
df['STATIONLINE'] = df['STATION'] + '_' + df['LINENAME']

In [ ]:
df['TIME'] = pd.to_datetime(df['TIME'])

In [ ]:
df.dtypes

In [ ]:
df['DATETIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'],format="%m/%d/%Y %H:%M:%S")

In [ ]:
df.DATETIME.head()

In [ ]:
(df
 .groupby(["KEY", "DATETIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head()

In [ ]:
# Explore the duplicate entries on 5/9
# mask = ((df["C/A"] == "G001") & 
# (df["UNIT"] == "R151") & 
# (df["SCP"] == "00-00-00") & 
# (df["STATION"] == "CONEY IS-STILLW") &
# # get datetime properties ~ .dt
# (df["DATETIME"].dt.date == datetime.datetime(2017, 5, 9).date()))
# df[mask].head()

In [ ]:
# Explore the duplicate entries on 4/28
# mask = ((df["C/A"] == "N329") & 
# (df["UNIT"] == "R201") & 
# (df["SCP"] == "00-00-00") & 
# (df["STATION"] == "WOODHAVEN BLVD") &
# # get datetime properties ~ .dt
# (df["DATETIME"].dt.date == datetime.datetime(2017, 4, 28).date()))
# df[mask].head()

In [ ]:
# df.sort_values(["KEY", "DATETIME"], inplace=True,
#               ascending = True)

In [ ]:
df.drop_duplicates(subset=["KEY", "DATETIME"], inplace=True)

In [ ]:
df.shape # two duplicates were removed

In [ ]:
df.sort_values(["KEY", "DATETIME"], inplace=True, ascending=True)

In [ ]:
# stationMax = df.groupby("STATION").ENTRIES.max()
# stationMax.sort_values(ascending=False)

In [ ]:
# df.sort_values(["KEY",'DATETIME'],inplace=True)

In [ ]:
df.head()

In [ ]:
df['ENTRYCOUNT'] = df.groupby("KEY")['ENTRIES'].diff()

In [ ]:
df = df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

df2 = df.copy()

In [ ]:
df.sort_values('ENTRYCOUNT', ascending=False).head(20)

In [ ]:
#df["ABSENTRY"] = df.ENTRYCOUNT.abs()

In [ ]:
#df.ABSENTRY.isnull

In [ ]:
sum(abs(df.ENTRYCOUNT) > 10000)/len(df)

In [ ]:
df.ENTRYCOUNT.hist()

In [ ]:
# df = df.drop(['EXITS','DESC'],axis=1,errors='ignore')
# df2 = df.copy()

In [ ]:
def clean_entries(row, max_counter):
    each = row["ENTRYCOUNT"]
    if each < 0:
        each = -each
    if each > max_counter:
        return 0
    return each

df2['ENTRYCOUNT'] = df2.apply(clean_entries, axis=1, max_counter=10000)

In [ ]:
df2StationMax = df2.groupby(['KEY','DATETIME']).ENTRYCOUNT.max()
df2StationMax.sort_values(ascending=False)

In [ ]:
# diffStationMin = df.groupby("STATION").ENTRYCOUNT.min()
# diffStationMin.sort_values(ascending=True)

In [ ]:
# stationMax = df.groupby("STATION").ENTRYCOUNT.max()
# stationMax.sort_values(ascending=False)

In [ ]:
stationDF = df2.groupby(["STATION"]).ENTRYCOUNT.sum().reset_index()
stationDF.sort_values('ENTRYCOUNT',ascending=False,inplace=True)

In [ ]:
stationDF.head(20)

In [ ]:
#stationDF.to_csv("stations.csv",index_label="Index")

In [ ]:
stationLineDF = df2.groupby(["STATIONLINE"]).ENTRYCOUNT.sum().reset_index()
stationLineDF.sort_values('ENTRYCOUNT',ascending=False,inplace=True)

In [ ]:
stationLineDF.to_csv("stationsandlines.csv",index_label="Index")

In [ ]:
top100 = stationDF[:100]

In [ ]:
len(top100) # check

In [ ]:
top100.head()

In [ ]:
len(stationDF)

In [ ]:
mid = stationDF[int(len(stationDF)/2):int((len(stationDF)/2)+5)]

In [ ]:
mid.head()

In [ ]:
# Replace Days with Name Days
df2['DAY']= df2.DATETIME.dt.dayofweek

In [ ]:
df2.DAY.value_counts()

In [ ]:
df2['NAMEDAY']=df2.DAY.replace({0:'1_Monday',1:'2_Tuesday', 2:'3_Wednesday', 3:'4_Thursday',
                       4:'5_Friday', 5:'6_Saturday',6:'7_Sunday'})

In [ ]:
df2.ENTRYCOUNT.isnull().sum()

In [ ]:
df2['HOUR']= df2.DATETIME.dt.hour

In [ ]:
# Drop Null Values
df2.dropna(subset=["ENTRYCOUNT"], axis=0, inplace=True)

In [ ]:
df2.shape

In [ ]:
stationDF = df2.groupby(["STATION",'LINENAME']).ENTRYCOUNT.sum().reset_index()
stationDF.sort_values('ENTRYCOUNT',ascending=False,inplace=True)

In [ ]:
stationDF.head()

In [ ]:
#top10.ENTRYCOUNT.sum()/df2.ENTRYCOUNT.sum()*100

In [ ]:
#top10.ENTRYCOUNT.sum()

In [ ]:
df2.STATION.nunique()

In [ ]:
top10 = stationDF[:10]

In [ ]:
top10.head(10)

## Plotting Penn Station

In [ ]:
top5 = stationDF[:5]

In [ ]:
sns.distplot(top5.ENTRYCOUNT)

In [ ]:
customFig = (10,8)

In [ ]:
topBar = top5.plot(kind='barh',x='STATION',sort_columns="ENTRYCOUNT", figsize=customFig)
topBar.invert_yaxis()

In [ ]:
pennDF = df2.loc[df2.STATION == '34 ST-PENN STA']

In [ ]:
pennDF.head()

In [ ]:
pennDayGroup = pennDF.groupby('NAMEDAY').ENTRYCOUNT.sum()
pennDayGroup

In [ ]:
pennDayGroup.plot(kind='line',figsize=customFig, linewidth=3.5)

In [ ]:
pennHourGroup = pennDF.groupby('HOUR').ENTRYCOUNT.sum()
pennHourGroup.plot(kind='line', figsize=customFig, linewidth = 3.5)

In [ ]:
pennHourGroup

In [ ]:
unionSquareDF = df2.loc[df2.STATION == '14 ST-UNION SQ']

In [ ]:
unionSquareDayGroup = unionSquareDF.groupby('NAMEDAY').ENTRYCOUNT.sum()

In [ ]:
twentyDayGroup.plot(kind='line',figsize=customFig, linewidth=3.5)

In [ ]:
unionSquareHourGroup = unionSquareDF.groupby('HOUR').ENTRYCOUNT.sum()
unionSquareHourGroup.plot(kind='line', figsize=customFig, linewidth=3.5)

In [ ]:
#pennDF.DATETIME.value_counts()

In [ ]:
# time_group = df2.groupby(pd.TimeGrouper(by='TIME',level='Time', freq='2H'))['ENTRYCOUNT'].agg('sum')   
# time_group.dropna(inplace=True)
# time_group = time_group.to_frame().reset_index()
# print(time_group)

In [ ]:
topNYCDF = df2.loc[(df2.STATION == 'GRD CNTRL-42 ST') | (df2.STATION == '34 ST-HERALD SQ') | 
                (df2.STATION == '42 ST-PORT AUTH') | (df2.STATION == '34 ST-PENN STA') |
                 (df2.STATION == '14 ST-UNION SQ')]

In [ ]:
topNYCDF.head()

In [ ]:
pairDF.shape

In [ ]:
topNYCDayGroup = pairDF.groupby(['STATION','NAMEDAY']).ENTRYCOUNT.sum().reset_index()
#pairDayGroup.plot(x='NAMEDAY', y='ENTRYCOUNT', figsize=(10,7))

In [ ]:
#topNYCDayGroup

In [ ]:
#pairDayGroup.plot(x='NAMEDAY', y='ENTRYCOUNT')
# ax = plt.subplot()
# sns.pointplot(x='DAY', y='ENTRYCOUNT', data=pairDayGroup,ax=ax, color='b')
# sns.pointplot(x='DAY', y='ENTRYCOUNT', data=pairDayGroup,ax=ax, color='r')
topNYCDayGroup.pivot('NAMEDAY','STATION','ENTRYCOUNT').plot(kind='line',figsize=customFig)

In [ ]:
topNYCHour = topNYCDF[['STATION','HOUR','ENTRYCOUNT']].copy()
topNYCHour.sort_values('ENTRYCOUNT', ascending=False,inplace=True)

In [ ]:
topNYCHour = (topNYCHour
 .sort_values(by=['STATION','ENTRYCOUNT'], ascending=[True,False])
 .groupby('STATION')
 .head(1))

In [ ]:
topNYCHour.plot

In [ ]:
topNYCHour.plot(kind='bar', figsize=customFig,x='HOUR',y="ENTRYCOUNT")

In [ ]:
topTechDF = df2.loc[(df2.STATIONLINE == '23 ST_FM') | (df2.STATIONLINE == '33 ST_6') | 
                (df2.STATIONLINE == '14 ST_123FLM') | (df2.STATIONLINE == '14 ST_ACEL') |
                 (df2.STATIONLINE == '23 ST_NRW')]


In [ ]:
topTechDayGroup = topTechDF.groupby(['STATIONLINE','NAMEDAY']).ENTRYCOUNT.sum().reset_index()
topTechDayGroup.pivot('NAMEDAY','STATIONLINE','ENTRYCOUNT').plot(kind='line',figsize=customFig)

In [ ]:
topTechDayGroupMax = topTechDF.groupby(['STATIONLINE','NAMEDAY']).ENTRYCOUNT.max()
topTechDayGroupMax

In [ ]:
topTechHour = topTechDF[['STATIONLINE','HOUR','ENTRYCOUNT']].copy()
topTechHour.sort_values('ENTRYCOUNT', ascending=False,inplace=True)

In [ ]:
topTechHour = (topTechHour
 .sort_values(by=['STATIONLINE','ENTRYCOUNT'], ascending=[True,False])
 .groupby('STATIONLINE')
 .head(1))
#topTechHour.groupby(['STATIONLINE','HOUR']).ENTRYCOUNT.max().head(1)

In [ ]:
topTechHour

In [ ]:
topTechDF.columns

In [ ]:
topTechHourDF = topTechDF.loc[(topTechDF.STATIONLINE == '23 ST_FM') & (topTechDF.DAY == 3)].copy()

In [212]:
topTechHourDF.groupby("TIME")

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,KEY,STATIONLINE,DATETIME,ENTRYCOUNT,DAY,NAMEDAY,HOUR
95001,N508,R453,00-00-00,23 ST,FM,IND,04/20/2017,00:00:00,237046617,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-20 00:00:00,435.0,3,4_Thursday,0
95002,N508,R453,00-00-00,23 ST,FM,IND,04/20/2017,04:00:00,237046634,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-20 04:00:00,17.0,3,4_Thursday,4
95003,N508,R453,00-00-00,23 ST,FM,IND,04/20/2017,08:00:00,237046656,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-20 08:00:00,22.0,3,4_Thursday,8
95004,N508,R453,00-00-00,23 ST,FM,IND,04/20/2017,12:00:00,237046850,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-20 12:00:00,194.0,3,4_Thursday,12
95005,N508,R453,00-00-00,23 ST,FM,IND,04/20/2017,16:00:00,237047314,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-20 16:00:00,464.0,3,4_Thursday,16
95006,N508,R453,00-00-00,23 ST,FM,IND,04/20/2017,20:00:00,237048653,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-20 20:00:00,1339.0,3,4_Thursday,20
95441,N508,R453,00-00-00,23 ST,FM,IND,04/27/2017,00:00:00,237061391,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-27 00:00:00,501.0,3,4_Thursday,0
95442,N508,R453,00-00-00,23 ST,FM,IND,04/27/2017,04:00:00,237061392,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-27 04:00:00,1.0,3,4_Thursday,4
95443,N508,R453,00-00-00,23 ST,FM,IND,04/27/2017,08:00:00,237061413,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-27 08:00:00,21.0,3,4_Thursday,8
95444,N508,R453,00-00-00,23 ST,FM,IND,04/27/2017,12:00:00,237061621,"N508,R453,00-00-00,23 ST",23 ST_FM,2017-04-27 12:00:00,208.0,3,4_Thursday,12


## Challenge 5

In [ ]:
turnstiles = df2.groupby(['C/A', 'UNIT', 'STATION','DATE']).ENTRYCOUNT.sum().reset_index()

In [ ]:
type(turnstiles)

In [ ]:
turnstiles.head()

## Challenge 6

In [ ]:
station = df.groupby(['STATION','DATE']).ENTRYCOUNT.sum().reset_index()


In [ ]:
station.head()

In [ ]:
station1 = station.loc[station.STATION == '1 AV']

In [ ]:
type(station1)

In [ ]:
# firstAveGroup = station.groupby('DATE')

In [ ]:
# firstAveGroup.plot(x=firstAveGroup.indices,y='ENTRYCOUNT')

## Challenge 7

In [ ]:
station1.plot(y='ENTRYCOUNT',x='DATE')

## Top 5 Stations based on Entries

In [ ]:
top5stations = df.groupby('STATION').ENTRYCOUNT.sum().reset_index()

In [ ]:
#top5stations = top5stations.sort_values('ENTRYCOUNT',ascending=False)

In [ ]:
top5stations.loc[top5stations.STATION == 'EASTCHSTER/DYRE']

In [ ]:
df.loc[df.STATION == 'EASTCHSTER/DYRE']

In [ ]:
df.ENTRYCOUNT.min()

In [ ]:
# Replace Days with Name Days
##titanic.sex=titanic.sex.replace({'female':0,'male':1})